In [1]:
import requests
import json
import pandas as pd

In [2]:
df = pd.read_csv('../data/external/upcoming-event.csv')
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Cory Sandhagen,TJ Dillashaw,-190.0,160.0,52.631579,160.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,330.0,700.0,700.0,900.0,130.0,330.0
1,Aspen Ladd,Macy Chiasson,-190.0,160.0,52.631579,160.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,165.0,275.0,600.0,1400.0,275.0,800.0
2,Kyler Phillips,Raulian Paiva,-275.0,220.0,36.363636,220.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,-110.0,450.0,800.0,1200.0,350.0,800.0
3,Darren Elkins,Darrick Minner,140.0,-165.0,140.000000,60.606061,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,330.0,165.0,550.0,250.0,900.0,800.0
4,Miranda Maverick,Maycee Barber,-135.0,115.0,74.074074,115.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,110.0,330.0,800.0,1100.0,1200.0,350.0


In [15]:
def cleaner(df):
    
    pattern_dif = r'_dif$'
    df_dif = df.filter(regex=pattern_dif)

    pattern_b = r'^B_'
    df_b = df.filter(regex=pattern_b)

    pattern_r = r'^R_'
    df_r = df.filter(regex=pattern_r)
    
    other_cols = df_dif.columns.append(df_b.columns).append(df_r.columns)

    df_info = df.drop(other_cols, axis=1)

    df2 = pd.concat([df_info, df_dif], axis=1)

    df3 = pd.concat([
        df2,
        df_b[['B_fighter', 'B_odds']],
        df_r[['R_fighter', 'R_odds']]
    ],
        axis=1)
    
    ufc = df3.drop(['finish_details', 'constant_1'], axis=1)

    ufc['date'] = pd.to_datetime(ufc['date']) # converting to dt

    # get month, day, year vals as objs
    ufc['year'] = ufc['date'].dt.year.astype('object')
    ufc['month'] = ufc['date'].dt.month.astype('object')
    ufc['day'] = ufc['date'].dt.day.astype('object')

    ufc.drop('date', axis=1, inplace=True) # drop orig col
    
    ufc.drop(['no_of_rounds', 'finish_round', 'total_fight_time_secs', 'finish', 'finish_round_time'], 
          axis=1, inplace=True) # removing after the fact cols
    
    ufc['weight_class'] = ufc['weight_class'].str.replace("Women\'s", "") # removing extra text since gender is given in col
    
    ufc['title_bout'] = ufc['title_bout'].astype('object') # changing to obj for consistency
    
    # set cutoffs for stds
    cuf_b = ufc['B_odds'].std() * 3
    cuf_r = ufc['R_odds'].std() * 3
    cuf_re = ufc['reach_dif'].std() * 3
    
    ufc = ufc[abs(ufc['reach_dif']) < cuf_re]
    
    ufc['empty_arena'] = ufc['empty_arena'].astype('object') # converting arena status to object
    
    for col in ufc.select_dtypes('object').columns:
        ufc[col] = ufc[col].apply(lambda x: str(x).strip()) # fixing whitespace issues
        
    # remove catchweight
    ufc = ufc[ufc['weight_class'] != 'Catch Weight']

    # make weight class ordinal
    ufc['weight_class'] = ufc['weight_class'].astype('category')
    ufc['weight_class']= ufc['weight_class'].cat.reorder_categories(['Strawweight', 'Flyweight', 'Bantamweight',
                                                                     'Featherweight', 'Lightweight', 'Welterweight',
                                                                     'Middleweight', 'Light Heavyweight', 'Heavyweight'],
                                                                    ordered=True)

    ufc.drop(['R_odds', 'win_dif'], axis=1, inplace=True)

    
    return ufc

In [16]:
cleaner(df)

ValueError: items in new_categories are not the same as in old categories